In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn import preprocessing 
import seaborn as sns

In [2]:
# Read in data
filepath = '/Users/maureenkeenan/Desktop/Kaggle/Titanic'
file_train = 'train.csv'
file_test = 'test.csv'
sample = pd.read_csv(os.path.join(filepath,file_train))
data = pd.read_csv(os.path.join(filepath,file_test))

In [3]:
# Explore data
sample.describe()
sample.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
y = sample.Survived # What we are fitting for
sample.groupby('Survived').Survived.count()

Survived
0    549
1    342
Name: Survived, dtype: int64

In [5]:
import nbimporter
import DataCleanup as dc

Importing Jupyter notebook from DataCleanup.ipynb


In [6]:
sample = dc.transform_features(sample)

DataCleanup.ipynb:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "    for n in range(len(Surname)):\n",


In [7]:
dc.encode_features(sample)

(     PassengerId  Survived  Pclass  \
 0              1         0       3   
 1              2         1       1   
 2              3         1       3   
 3              4         1       1   
 4              5         0       3   
 5              6         0       3   
 6              7         0       1   
 7              8         0       3   
 8              9         1       3   
 9             10         1       2   
 10            11         1       3   
 11            12         1       1   
 12            13         0       3   
 13            14         0       3   
 14            15         0       3   
 15            16         1       2   
 16            17         0       3   
 17            18         1       2   
 18            19         0       3   
 19            20         1       3   
 20            21         0       2   
 21            22         1       2   
 22            23         1       3   
 23            24         1       1   
 24            25        

In [8]:
# Do a quick subset of features
quick_features = ['Sex','Age','Pclass','SibSp','Cabin','TitleKey','Fare','Embarked']
X = sample.drop(columns = ['Survived'])
y = sample.Survived

In [9]:
# Define subfeatures
features_1 = ['Sex','Age','Pclass']
features_2 = ['Sex','Age','Pclass','SibSp','Parch']
features_3 = ['Sex','Age','Pclass','Family']
features_4 = ['Sex','Age','Pclass','SibSp','Parch','Fare']
features_5 = ['Sex','Age','Pclass','SibSp','Parch','Fare','Title']

features = [features_1, features_2, features_3, features_4, features_5]

In [22]:
# Break off validation set from training data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=0)

print(len(X_train))

712


In [27]:
# import the class
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

# Define the models
model = RandomForestClassifier(n_estimators=100, criterion='mae', random_state=0)

for i in range(0, len(features)):
    model.fit(X_train[features[i]], y_train)
    y_pred = model.predict(X_test[features[i]])
    
    # get stats
    f1 = metrics.f1_score(y_test,y_pred, pos_label=1)
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
    print("Model ",i+1,"F1: ", f1.round(2), " CNF: \n", cnf_matrix)
    

KeyError: 'mae'

In [12]:
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred).round(2))
# print("Precision:",metrics.precision_score(y_test, y_pred).round(2))
# print("Recall:",metrics.recall_score(y_test, y_pred).round(2))

In [24]:
differences = y_pred-y_test
FN = differences[differences == -1].index
#sample.iloc[FN,:]
FP = differences[differences == 1].index
#sample.iloc[FP,:]

In [25]:
# Do this for the test set

#dc.transform_features(data)
#dc.encode_features(data)
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,S


In [26]:
data_predict = data[features_3]
data_predict.head()

KeyError: "['Family'] not in index"

In [ ]:
model.fit(X_train[features_3], y_train)
predictions = model.predict(data_predict)


In [ ]:
# Save into output file
ids = data['PassengerId']

output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
output.to_csv('titanic-predictions_RF.csv', index = False)
output.head()